In [3]:
# from core import load_metrics, generate_population, tournament_selection, fixed_crossover, mutate, replace_weakest
from core.utils import load_metrics, generate_population, tournament_selection
from core.crossover import fixed_crossover, fixed_crossover_twopoint , ordered_crossover, partialMap_crossover
from core.mutation import mutate, multiple_mutate, inversion, scramble
from core.replacement import replace_firstweak, replace_weakest

In [12]:
sample = [
    [0,2,1,2,1],
    [2,0,2,2,2],
    [1,2,0,2,1],
    [2,2,2,0,2],
    [1,2,1,2,0]
]

### Chromosome Class

In [25]:
from core import fitness
from typing import List

class Chromosome:
    def __init__(self, genes:List[int]):
        self.genes = genes
        self.phenomes = self.__fitness__(genes=genes)

    def __fitness__(self, genes:List[int]) -> int:
        return fitness(genes, distance_metric=sample)

    def __str__(self):
        return f"Genes: {self.genes}\nPhenomes: {self.phenomes:.5f}"


### Experiment01